#Import Libraries

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 78.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 95.2 MB/s eta 0:00:00


In [ ]:
from sklearn.svm import SVC
from transformers import BertTokenizer, BertModel
import torch
import pandas as pd

#Import Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Sarcasm/dataset/isarcasm/clean/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Sarcasm/dataset/isarcasm/clean/test.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#BERT Embeddings

In [ ]:
# Load pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Function to get BERT embeddings for text
def get_bert_embeddings(text):
    # Tokenize text
    input_ids = tokenizer.encode(text, return_tensors='pt')
    # Get BERT embeddings
    with torch.no_grad():
        outputs = model(input_ids)
    # Return the last hidden state of the first token (CLS token)
    return outputs[0][:,0,:].numpy()

In [ ]:
# Get BERT embeddings for train and test data
X_train = [get_bert_embeddings(text).reshape(-1) for text in train['tweet']]
X_test = [get_bert_embeddings(text).reshape(-1) for text in test['tweet']]

# Get labels for train data
y_train = train['sarcastic']

#SVM Classifier

In [ ]:
# Create SVM classifier with rbf kernel and regularization value 10
clf = SVC(kernel='rbf', C=10)

In [ ]:
# Fit classifier on train data
clf.fit(X_train, y_train)

SVC(C=10)

In [ ]:
# Predict on test data
y_pred = clf.predict(X_test)

#Report

In [ ]:
from sklearn.metrics import classification_report

# Get true labels for test data
y_test = test['sarcastic']

# Print classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.97      0.91      1194
           1       0.37      0.11      0.17       200

    accuracy                           0.85      1394
   macro avg       0.62      0.54      0.54      1394
weighted avg       0.80      0.85      0.81      1394

